In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
%matplotlib inline

In [2]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# pd.set_option('display.max_colwidth', -1)

## Importing data

Importing a sample of the dataset:

In [3]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [4]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
                      quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

19:02:13 Starting to read file... (3 min)


C:\Users\Daniel\Anaconda3\envs\ADA-kernel\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,5,6,8,9,10,11,12,13,14,15,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


19:04:52 File is loaded!


In [5]:
tw.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,1.7341e+07,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508.0,9016.0,"Bern, Switzerland"
1,9514846412,7.19828e+06,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703.0,4687.0,"Zurich, Switzerland"


## Extrating hashtags

In [6]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [7]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

19:04:52 Making hashtags... (2 min)
19:06:40 Done.


In [8]:
twh.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
8,9519737890,1.46579e+07,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,...,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
10,9521789689,9.96202e+06,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,...,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


## Cleaning data and making date index

In [9]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 3875280 tweets to 3875280 tweets.


In [10]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

19:06:43 Removing bad dates...
19:06:45 Finished.


In [11]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

19:06:45 Starting to examine dates...
19:06:48 There are 0 dates that cannot be transformed.


In [12]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

19:06:48 Starting copy...
19:06:48 Converting to datetime...
19:06:52 Setting up new indices...
19:06:52 Deleting old "createdAt" column...
19:06:52 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,1.46579e+07,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
2010-02-23 11:28:27,9521789689,9.96202e+06,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


In [13]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [14]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [15]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

19:06:52 Multiplying the hashtag rows... (around 10 min)
19:16:05 Finished! 3350600 rows will be added to the dataframe!


In [16]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

19:16:05 Starting to make the new dataframe with additionnal rows..
19:16:20 Starting to append the two df... Old df size = 3875280
19:16:27 Done! New df size = 7225880


In [17]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,1121,763.0,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,47.5356,9.14004,eLd0raDo,14,Tweetbot for iOS,http://tapbots.com/tweetbot,6735.0,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2012-07-29 16:15:54,1008,767.0,vfb,229611214802669570,NaN,47.6683,9.48077,7e00050f2f9230bc,47.6684,9.47092,jens_nagler,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,1697.0,Man sollte ja nicht zu viel reininterpretieren...,NaN,5.71385e+07,Esslingen,Jens Nagler
2012-07-29 16:21:45,1008,767.0,vfb,229612686525231104,2.296124e+17,47.6682,9.48068,7e00050f2f9230bc,47.6684,9.47092,jens_nagler,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,1697.0,"@LLcurly Das Besondere daran ist, dass nur zwe...",NaN,5.71385e+07,Esslingen,Jens Nagler


In [18]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Aggregation of rows by date

In [19]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [20]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

In [21]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [22]:
def aggDate(df):
    groupedDf = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : lambda x: x.iloc[0], ## the first occurence
                                                          'numberOfTweets' : 'count'})
    return groupedDf

In [23]:
pr('Making column number of tweets')
tw6['numberOfTweets'] = 1
pr('Starting group by hastag...')
gp = tw6.groupby('hashtag')
pr('Starting to put hashtag in dictionary... (around 50 min)')

count = 0
lengp = len(gp)
printingValue = int(lengp / 10)
dictionary = {}
for hashtag, df in gp:
    dictionary[hashtag] = aggDate(df)
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/lengp*100))
pr('Finished operations! Dictionary with {} different hashtags.'.format(len(dictionary)))

19:16:50 Making column number of tweets
19:16:50 Starting group by hastag...
19:16:50 Starting to put hashtag in dictionary... (around 50 min)
19:22:12 10%
19:27:17 20%
19:32:07 30%
19:37:02 40%
19:41:52 50%
19:46:45 60%
19:51:35 70%
19:56:21 80%
20:00:59 90%
20:05:33 100%
20:05:33 Finished operations! Dictionary with 607601 different hashtags.


In [24]:
# aggDate(gp.get_group('twibisg')).head(3)
# dictionary['twibisg'].head(3)
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,numberOfTweets,hashtag,latitude,text,longitude
2016-04-22,1,pallino,45.8259,Hip thruster! #pallino #rehab #teaching #worki...,8.80399
2016-05-04,1,pallino,45.8361,At work on the court! #rehab #pallino #conditi...,8.80239


## Starting event detection

In [25]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 5 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 2 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.

In [26]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
            df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

20:05:33 Starting to compute 607601 dict items to detect event. (4 min)
20:05:55 10%
20:06:17 20%
20:06:39 30%
20:07:01 40%
20:07:23 50%
20:07:45 60%
20:08:22 70%
20:08:44 80%
20:09:06 90%
20:09:28 100%
20:09:28 Finished! Number of events detected = 22109


## Making single event dataframe

In [27]:
eventRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        eventRowsList.append(rowToAdd)

In [28]:
eventRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df.apply(applyToMakeEventDf, axis=1)

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

20:09:28 Starting to make event df with 607601 dataframes. (around 6 min)
20:10:01 10%
20:10:34 20%
20:11:07 30%
20:11:39 40%
20:12:12 50%
20:12:45 60%
20:13:18 70%
20:13:50 80%
20:14:23 90%
20:14:56 100%
20:14:56 Making new dataframe.
20:14:56 Finished! Dataframe with 22109 rows


In [29]:
print('Events dataframe:')
new_events.head(10)

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2014-06-15,schweizernati,47.40400,8.269810,6,Röstigraben in der Nati? Spielen die gegeneina...
2015-01-25,tsipras,45.80550,8.487300,7,Gli opinionisti potrebbero almeno aspettare i ...
2015-01-26,tsipras,46.10000,8.768430,6,Ma... tu quoque #Tsipras? Sei durato solo una ...
2015-11-24,popporytravel2015,46.68960,7.864830,26,#Switzerland #icebergluxurytrip #popporytravel...
2015-06-17,wave,47.31180,7.929100,6,1. Stop der Wave in Oftringen #Wave #wavetroph...
2012-12-01,etg,46.15015,6.332685,6,http://t.co/PuLhz9V5 \nDans la plaaace en vip ...
2013-04-17,etg,45.91635,6.134280,8,"@mliinaviollet je go a Annecy\n#etg-psg✌_$$$_""..."
2013-05-08,etg,45.91650,6.118450,7,Allez les roses !! #etg #etgfcl #coupedefrance...
2013-05-16,etg,45.91665,6.118855,6,#etg #alpeslemanangels #annecy #stade avec @ye...


In [30]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,numberOfTweets,hashtag,latitude,text,longitude,event
2012-10-16,1,schweizernati,46.94700,Die Schweibanische Nationalmannschaft auf dem ...,7.395830,False
2013-10-11,2,schweizernati,47.08780,wünsche der #schweizernati rund um @GokhanInle...,7.530045,False
2013-10-15,1,schweizernati,46.96310,Enjoing tha Game... #soccer #schweizernati #br...,7.464800,False
2014-06-12,1,schweizernati,47.18250,freu mi uf 4woche biertrinken&amp;übermüdet go...,8.456750,False
2014-06-14,1,schweizernati,47.41080,#SchweizerNati #Celeste #633 @ Glatt Zentrum h...,8.568150,False
2014-06-15,6,schweizernati,47.40400,Röstigraben in der Nati? Spielen die gegeneina...,8.269810,True
2014-06-16,1,schweizernati,47.37520,2-1 bravooooooooooooooo #schweizernati #suisse...,8.530380,False
2014-06-20,2,schweizernati,47.20710,@SFV_ASF wäre schön wenn wir heute noch Fussba...,8.385155,False
2014-06-25,2,schweizernati,46.80875,@cathfoe Irren ist menschlich... :) freuen wir...,7.631495,False
2014-07-01,5,schweizernati,47.20160,@SFV_ASF das war ein SUPERSPIEL #srfbrasil #SU...,7.896540,False


## Exporting data

In [69]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,schweizernati,47.404,8.26981,6,Röstigraben in der Nati? Spielen die gegeneina...,2014-06-15


We are going to generate the right datetimes for the jsons:

In [70]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [71]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [72]:
pr('Converting dates...')
e_df['year'] = e_df['date'].apply(lambda x: x.year)
e_df['month'] = e_df['date'].apply(lambda x: x.month)
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)
pr('Done.')
e_df.head(1)

01:20:53 Converting dates...
01:21:00 Done.


,hashtag,latitude,longitude,numberOfTweets,text,date,year,month,utc_date,unix_time
0,schweizernati,47.404,8.26981,6,Röstigraben in der Nati? Spielen die gegeneina...,2014-06-15,2014,6,1402790400000,1401580800000


Grouping by months and generating the dictionary for the json.

In [73]:
e_gb_month = e_df.groupby(e_df.unix_time)

In [75]:
pr('Making event list...')
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}
pr('Done.')

01:21:52 Making event list...
01:22:13 Done.


In [77]:
exportFilename = 'export_twitter_events_' + datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S") + '.json'
exportPath =  os.path.join('data', exportFilename)

pr('Exporting to json...')
with open(exportPath, 'w') as f:
     json.dump(final_events, f)
pr('Export done. File "{}" has been created.'.format(exportFilename))

01:22:27 Exporting to json...
01:22:29 Export done. File "export_twitter_events_2017-01-30_01h22min27.json" has been created.
